# Working with Unknown Dataset Sizes

This notebook demonstrates the features built into OpenDP to handle unknown or private dataset sizes.

### Load exemplar dataset

In [1]:
import os
data_path = os.path.join('.', 'data', 'PUMS_california_demographics_1000', 'data.csv')
with open(data_path) as data_file:
    data = data_file.read()

By looking at the private data, we see this dataset has 1000 observations (rows).
Oftentimes the number of observations is public information.
For example, a researcher might run a random poll of 1000 respondents and publicly announce the sample size.

However, there are cases where simply the number of observations itself can leak private information.
For example, if a dataset contained all the individuals with a rare disease in a community,
then knowing the size of the dataset would reveal how many people in the community had that condition.
In general, any given dataset may be some well-defined subset of a population.
The given dataset's size is equivalent to a count query on that subset,
so we should protect the dataset size just as we would protect any other query we want to provide privacy guarantees for.

OpenDP assumes the sample size is private information.
If you know the dataset size (or any other parameter) is publicly available,
then you are free to make use of such information while building your measurement.

OpenDP will not assume you truthfully or correctly know the size of the dataset.
Moreover, OpenDP cannot respond with an error message if you get the size incorrect;
doing so would permit an attack whereby an analyst could repeatedly guess different dataset sizes until the error message went away,
thereby leaking the exact dataset size.

If we know the dataset size, we can incorporate it into the analysis as below,
where we provide `size` as an argument to the release of a sum on age.
While the "sum of ages" is not a particularly useful statistic, it's plenty capable of demonstrating the concept.

In [2]:
from opendp.trans import *
from opendp.meas import make_base_geometric
from opendp.mod import enable_features
enable_features("contrib")

# Define parameters up-front
# Each parameter is either a guess, a DP release, or public information
var_names = ["age", "sex", "educ", "race", "income", "married"]  # public information
size = 1000 # public information
age_bounds = (0, 100) # an educated guess
constant = 38 # average age for entire US population (public information)

dp_sum = (
    # Load data into a dataframe of string columns
    make_split_dataframe(separator=",", col_names=var_names) >>
    # Selects a column of df, Vec<str>
    make_select_column(key="age", TOA=str) >>
    # Cast the column as Vec<Int>
    make_cast(TIA=str, TOA=int) >>
    # Impute missing values to 0
    make_impute_constant(constant) >>
    # Clamp age values
    make_clamp(bounds=age_bounds) >>
    # Resize with the known `size`
    make_bounded_resize(size=size, bounds=age_bounds, constant=constant) >>
    # Aggregate
    make_sized_bounded_sum(size=size, bounds=age_bounds) >>
    # Noise
    make_base_geometric(scale=1.)
)

release = dp_sum(data)
print("DP sum:", release)


DP sum: 44798


### Providing incorrect dataset size values

However, if we provide an incorrect value of `n` we still receive an answer.

`make_sum_measurement` is just a convenience constructor for building a sum measurement from a `size` argument.

In [3]:
preprocessor = (
    make_split_dataframe(separator=",", col_names=var_names) >>
    make_select_column(key="age", TOA=str) >>
    make_cast_default(TIA=str, TOA=int) >>
    make_clamp(age_bounds)
)

def make_sum_measurement(size):
    return make_bounded_resize(size=size, bounds=age_bounds, constant=constant) >> \
           make_sized_bounded_sum(size=size, bounds=age_bounds) >> \
           make_base_geometric(scale=1.0)

lower_n = (preprocessor >> make_sum_measurement(size=200))(data)
real_n = (preprocessor >> make_sum_measurement(size=1000))(data)
higher_n = (preprocessor >> make_sum_measurement(size=2000))(data)

print("DP sum (n=200):  {0}".format(lower_n))
print("DP sum (n=1000): {0}".format(real_n))
print("DP sum (n=2000): {0}".format(higher_n))



DP sum (n=200):  8809
DP sum (n=1000): 44796
DP sum (n=2000): 82799


### Analysis with no provided dataset size
If we do not believe we have an accurate estimate for `size` we can instead pay some of our privacy budget
to estimate the dataset size.
Then we can use that estimate in the rest of the analysis.
Here is an example:

In [4]:
# First, make the measurement
dp_count = (
    make_split_dataframe(separator=",", col_names=var_names) >>
    make_select_column(key="age", TOA=str) >>
    make_count(TIA=str) >>
    make_base_geometric(scale=1.)
)
dp_count_release = dp_count(data)
print("DP count: {0}".format(dp_count_release))

dp_sum = preprocessor >> make_sum_measurement(dp_count_release)
dp_sum_release = dp_sum(data)
print("DP sum: {0}".format(dp_sum_release))

DP count: 1000
DP sum: 44795


Note that our privacy usage has increased because we apportioned some epsilon for both the release count of the dataset,
and the mean of the dataset.

### OpenDP `resize` vs. other approaches
The standard formula for the mean of a variable is:
$\bar{x} = \frac{\sum{x}}{n}$

The conventional, and simpler, approach in the differential privacy literature, is to: 

1. compute a DP sum of the variable for the numerator
2. compute a DP count of the dataset rows for the denominator
3. take their ratio

This is sometimes called a 'plug-in' approach, as we are plugging-in differentially private answers for each of the
terms in the original formula, without any additional modifications, and using the resulting answer as our
estimate while ignoring the noise processes of differential privacy. While this 'plug-in' approach does result in a
differentially private value, the utility here is generally lower than the solution in OpenDP.  Because the number of
terms summed in the numerator does not agree with the value in the denominator, the variance is increased and the
resulting distribution becomes both biased and asymmetrical, which is visually noticeable in smaller samples.

We have noticed that for the same privacy loss,
the distribution of answers from OpenDP's resizing approach to the mean is tighter around the true dataset value (thus lower in error) than the conventional plug-in approach.

*Note, in these simulations, we've shown equal division of the epsilon for all constituent releases,
but higher utility (lower error) can be generally gained by moving more of the epsilon into the sum,
and using less in the count of the dataset rows, as in earlier examples.*